In [44]:
import requests
import re

url = 'http://www.calbears.com/roster.aspx?path=mbball'
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36'
headers = {'User-Agent': user_agent}
r = requests.get(url, headers=headers)

In [45]:
def findHeight(line):
    h = re.search("<span>[1-7]{1,2}'[1-9]{1,2}&quot", line)
    return h

In [46]:
def findWeight(line):
    w = re.search("<span>[0-9]{3}\slbs", line)
    return w
    

In [47]:
def findImage(line):
    img = re.search("images.2016.8.22.[a-zA-Z_0-9]*.JPG", line)
    return img

In [48]:
all_data = []
def findAll():
    page = r
    count = 0
    prevLine = " "
    imgFound = False
    heightFound = False
    weightFound = False
    for line in r.iter_lines():
        if(count == 0):
            prevLine = str(line.decode("ISO-8859-1"))
            count = count + 1
            pass
        if(count!= 0):
            lineToUse = str(line.decode("ISO-8859-1"))
            if(imgFound == False):
                im = findImage(prevLine + lineToUse)
                if(im != None):
                    imgFound = True
                    image = im.group(0)
                else:
                    prevLine = lineToUse
            if(imgFound == True):
                if(heightFound == False):
                    h = findHeight(lineToUse)
                    if(h != None):
                        heightFound = True
                        height = h.group(0)
                if(heightFound == True and weightFound == False):
                    w = findWeight(lineToUse)
                    if(w != None):
                        weight = w.group(0)
                        data = {'url': url,
                                'height': height, 'weight': weight, 'image': image}
                        all_data.append(data)
                        imgFound = False
                        heightFound = False
                        weightFound = False

In [50]:
#go through array of objects and clean up data
import json
for x in range(0, len(all_data)):
    h = re.search("[1-7]{1}'[1-9]{1,2}", all_data[x]['height']) 
    all_data[x]['height'] = {
      'amount': str(h.group(0)),
      'unit': "cm"
    }
    
    w = re.search("[0-9]{1,3}\slbs", all_data[x]['weight'])
    all_data[x]['weight'] = {
        'amount': str(w.group(0)).split()[0],
        'unit': "kg"
    }
    all_data[x]['image'] = "http://www.calbears.com/" + all_data[x]['image']

print(json.dumps(all_data, indent=4, sort_keys=True))

[]


In [39]:
def convertMetrics():
    for x in range(0, len(all_data)):
        weight = all_data[x]['weight']['amount']
        all_data[x]['weight']['amount'] = float(weight) * .453
        h = re.split("[']", all_data[x]['height']['amount'])
        height = (int(h[0]) * 12 + int(h[1])) * 2.54
        all_data[x]['height']['amount'] = height

In [40]:
print(json.dumps(all_data, indent=4, sort_keys=True))

[]


In [41]:
def calcBMI(height, weight):
    height = height / 100
    bmi = weight/(height * height)
    return bmi

In [42]:
import urllib
import urllib.request
from urllib.request import urlretrieve
from PIL import Image
import random
import cv2
def imagesToFolder():
    count = 0
    for x in range(0, len(all_data)):
        try:
            height = int(all_data[x]['height']['amount'])
            weight = int(all_data[x]['weight']['amount'])
            if(len(str(weight)) > 1): 
                count = count + 1
                bmiNum = calcBMI(height, weight)
                url = all_data[x]['image']
                print(weight)
                nameOfImage = "B-"+ str(bmiNum) + "H-" + str(height) + "W-" + str(weight) + "-" + str(hash(random.random())) + "6_8.jpg"
                path = "/Users/nikki-genlife/Desktop/anaconda/all_usable_imageNames"
                response = urllib.request.urlopen(url)
                y = response.read()
                fd = open(path + "/" + nameOfImage, "wb")
                fd.write(y)
                fd.close()
            else:
                pass
        except urllib.error.HTTPError:
            pass